In [12]:
"""
Script de Treinamento de Modelos de Machine Learning para Análise de Sentimentos

Este script realiza o treinamento e avaliação de vários modelos de machine learning para a análise de sentimentos em textos. 
Utiliza uma abordagem de vetorização de texto (TF-IDF) e diferentes algoritmos de classificação como Random Forest, Regressão Logística e Gradient Boosting para identificar o sentimento expresso nos textos. 
O processo inclui carregar dados, preparar os textos e rótulos, criar pipelines de processamento, realizar busca de hiperparâmetros com validação cruzada, e avaliar os modelos com métricas como acurácia e relatório de classificação.

Funcionalidades:
- Carregamento de dados de um arquivo CSV.
- Preparação e codificação dos textos e rótulos.
- Criação de pipelines de machine learning para vetorização e classificação.
- Busca de hiperparâmetros utilizando RandomizedSearchCV para otimizar os modelos.
- Avaliação dos modelos utilizando acurácia e relatório de classificação detalhado.
- Salvamento dos modelos com melhor desempenho.

Dependências:
- Pandas e NumPy para manipulação de dados.
- Scikit-learn para criação de pipelines, modelos de classificação, busca de hiperparâmetros e avaliação.
- Joblib para salvar os modelos treinados.

Membros do grupo:

Alonso Batista de Oliveira Júnior
André Moreira de Carvalho
Gustavo Castro Candeia
Halex Maciel Silva Vieira
Welbert Luiz Silva Junior

"""
import pandas as pd
import numpy as np
from typing import Tuple, Dict, Any
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix, cohen_kappa_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.exceptions import NotFittedError
import joblib
import logging
from tabulate import tabulate
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

In [13]:
# Definição de constantes para o caminho dos arquivos
DATA_FILE_PATH = '../data/cleaned_dataset.csv'
MODEL_DIR = '../models'
LOG_DIR = '../logs'

In [14]:
# Save the log to a file in the logs directory
logging.basicConfig(filename=f'{LOG_DIR}/training.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [15]:
MODEL_CONFIGS: Dict[str, Dict[str, Any]] = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'clf__n_estimators': [300, 500],
            'clf__max_depth': [30, 50]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingClassifier(random_state=42),
        'params': {
            'clf__n_estimators': [300, 500],
            'clf__learning_rate': [0.01, 0.001]
        }
    }
}

In [16]:
def load_data(filepath: str) -> pd.DataFrame:
    try:
        return pd.read_csv(filepath, usecols=['Clean_Text', 'Label'])
    except FileNotFoundError as e:
        logging.error(f"Arquivo não encontrado: {filepath}")
        raise e

In [17]:
def prepare_data(df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, LabelEncoder]:
    """Prepara os dados para treinamento, retornando conjuntos de treino e teste, e o LabelEncoder."""
    df['Clean_Text'] = df['Clean_Text'].fillna('').astype(str)
    texts = df['Clean_Text'].values
    
    encoder = LabelEncoder()
    labels = encoder.fit_transform(df['Label'])
    
    # Imprime os labels e seus códigos
    print("Classes codificadas e seus códigos:")
    for label, code in zip(encoder.classes_, range(len(encoder.classes_))):
        print(f"Label '{label}' é codificado como {code}")
    print("")
    
    return train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels), encoder

In [18]:
def create_pipeline(model: Any) -> Pipeline:
    return Pipeline([
        ('tfidf', TfidfVectorizer(max_features=2000)),
        ('clf', model)
    ])

In [19]:
def additional_metrics(y_test: np.ndarray, predicted_probabilities: np.ndarray, predictions: np.ndarray) -> Dict[str, Any]:
    conf_matrix = confusion_matrix(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='weighted')
    kappa = cohen_kappa_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, predicted_probabilities, multi_class='ovr', average='weighted')
    
    return {
        "confusion_matrix": conf_matrix,
        "f1_score": f1,
        "kappa_score": kappa,
        "roc_auc_per_class": roc_auc
    }

In [20]:
def evaluate_model(model: Pipeline, X_test: np.ndarray, y_test: np.ndarray) -> Dict[str, Any]:
    try:
        predictions = model.predict(X_test)
        predicted_probabilities = model.predict_proba(X_test)  # Assegurar que estamos usando predict_proba para ROC AUC

        base_metrics = {
            'classification_report': classification_report(y_test, predictions, output_dict=True),
            'accuracy': accuracy_score(y_test, predictions),
            'roc_auc': roc_auc_score(y_test, predicted_probabilities, multi_class='ovr', average='weighted')
        }

        add_metrics = additional_metrics(y_test, predicted_probabilities, predictions)  # Passar predicted_probabilities aqui também
        base_metrics.update(add_metrics)
        return base_metrics
    except NotFittedError as e:
        logging.error("Modelo não ajustado")
        raise e
    except Exception as e:
        logging.error(f"Erro ao avaliar o modelo: {e}")
        raise e


In [21]:
def main() -> None:
    df = load_data(DATA_FILE_PATH)
    (X_train, X_test, y_train, y_test), label_encoder = prepare_data(df)
    joblib.dump(label_encoder, f'{MODEL_DIR}/label_encoder.joblib')

    model_performances = []

    for name, config in tqdm(MODEL_CONFIGS.items(), desc="Treinando modelos"):
        pipeline = create_pipeline(config['model'])
        search = RandomizedSearchCV(pipeline, config['params'], cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
        search.fit(X_train, y_train)
        metrics = evaluate_model(search.best_estimator_, X_test, y_test)
        logging.info(f"Relatório de classificação para {name}:\n{metrics['classification_report']}")
        model_performances.append((name, metrics['accuracy'], metrics['roc_auc'], search.best_estimator_))
        logging.info(f"Melhores hiperparâmetros para {name}: {search.best_params_}")
        logging.info(f"Acurácia para {name}: {metrics['accuracy']:.4f}")
        logging.info(f"ROC-AUC para {name}: {metrics['roc_auc']:.4f}")
        logging.info(f"Melhor modelo para {name}: {search.best_estimator_}\n")

    model_performances.sort(key=lambda x: x[1], reverse=True)
    for i, (name, acc, auc, model) in enumerate(model_performances, 1):
        model_path = f'{MODEL_DIR}/top_{i}_{name}_model.joblib'
        joblib.dump(model, model_path)
        logging.info(f"Modelo {name} salvo com acurácia de {acc:.4f} em {model_path}")


In [22]:
if __name__ == "__main__":
    main()

Classes codificadas e seus códigos:
Label 'litigious' é codificado como 0
Label 'negative' é codificado como 1
Label 'positive' é codificado como 2
Label 'uncertainty' é codificado como 3



Treinando modelos:   0%|          | 0/2 [00:00<?, ?it/s]

Fitting 3 folds for each of 4 candidates, totalling 12 fits


Treinando modelos:  50%|█████     | 1/2 [56:07<56:07, 3367.28s/it]

Fitting 3 folds for each of 4 candidates, totalling 12 fits


Treinando modelos: 100%|██████████| 2/2 [4:32:50<00:00, 8185.20s/it]
